In [15]:
from dotenv import load_dotenv
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chat_models import ChatOpenAI
from pinecone import Pinecone
from pinecone import ServerlessSpec
from langchain_core.documents import Document
import uuid
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter



In [3]:
load_dotenv()

os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY", "")

In [4]:
llm = ChatOpenAI(temperature=0)


C:\Users\Adari Shanmukh\AppData\Local\Temp\ipykernel_2784\650358916.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)


In [5]:
pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY"),
)

In [6]:

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings.embed_query("hello AI")



[-0.033388204872608185,
 0.03453974053263664,
 0.05947452038526535,
 0.05928610637784004,
 -0.06353536248207092,
 -0.06819581985473633,
 0.08823318779468536,
 0.03444082289934158,
 -0.032785143703222275,
 -0.01581503450870514,
 0.020981701090931892,
 -0.01834029145538807,
 -0.039832133799791336,
 -0.08047077804803848,
 -0.014469215646386147,
 0.0332648828625679,
 0.01425927970558405,
 -0.03404996171593666,
 -0.142915740609169,
 -0.023083269596099854,
 -0.02138008177280426,
 0.002633549040183425,
 -0.04729273170232773,
 -0.010752758011221886,
 -0.06866797059774399,
 0.031125066801905632,
 0.0759459063410759,
 0.0011282657505944371,
 0.011631992645561695,
 -0.036039240658283234,
 0.0448375940322876,
 0.018390778452157974,
 0.12672795355319977,
 -0.0013597294455394149,
 0.008206664584577084,
 0.06909965723752975,
 -0.08076354116201401,
 -0.05841318145394325,
 0.05375447869300842,
 0.02622760273516178,
 -0.006828595884144306,
 -0.05635837838053703,
 0.003292993875220418,
 -0.07250176370143

In [7]:



# Managed by cloud provider
if not pc.has_index("my-index1"):
    pc.create_index(
        name="my-index1",
        dimension=384,
        metric="cosine",
        spec=ServerlessSpec
        (
            cloud="aws",
            region="us-east-1"
        )
    )


In [8]:
pc.has_index("my-index1")
index = pc.Index("my-index1")

In [9]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index,embedding=embeddings)


In [10]:
FILE_PATH=r"D:\PROJECT_PRACTISE_DIRS\AGENTICAI\2-Langchain_Basics\2.2-DataTransformer\syllabus.pdf"
loader=PyPDFLoader(FILE_PATH)


In [18]:
len(loader.load())
pages = []
async for page in loader.alazy_load():
    pages.append(page)

In [19]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,#hyperparameter
    chunk_overlap=50 #hyperparemeter
)
documents = splitter.split_documents(pages)

In [20]:
uuids=[]
for i in range(len(documents)):
    uuids.append(str(uuid.uuid4()))


In [13]:
uuids

['bb2efa89-de31-426e-9df3-729efce95996',
 '08da3ea5-77a4-4c1c-a3df-6038ffc9ebc6',
 '0a0f11fa-170c-4101-9e2d-cf1d6be54a60',
 '665d4688-a988-41fe-8688-fa5b4e989e30',
 'aeb28763-2ef8-4cfb-a94e-72d5330158a8',
 'dd6293bd-2350-489f-8092-9981adb256e7',
 'f73f4c9e-ba26-4f4c-acc1-c30badb2f10a',
 'd6d21618-7ba9-489e-8a97-1edfd98b8454',
 'fe0dfb14-9fee-4766-9dec-d24ee62c7567',
 '38cacfe0-17b8-427f-930a-ef1151277f24',
 'c63fd7a1-3a3f-4064-b960-a45658e55a69',
 '2c6d8146-0f9c-46b8-8a80-66ed4af83286',
 'd0f85c40-7d84-4fe0-b91b-49128beae8a2',
 'b7d30ff1-1590-4b92-a342-f0969814a192',
 '9f5a8043-5430-4652-8a12-d45d6e56f056',
 'd62fa480-d2ef-4042-8ef9-a3de20e6900b',
 '88c81cd9-9bcc-427f-b3aa-ac83db61ca2a',
 '01226784-4c07-48f2-a37d-8df870986bea',
 '284c46f5-19a3-475f-9fa3-0b43853fafba',
 '2447d80e-4396-435e-987d-f105c77a47b3',
 '47117ef3-53aa-43ed-9800-ea042f0db17b',
 'b63bf794-0c5c-42f1-a730-5ebae1a12f02',
 '33c15129-e8ea-4816-8a9d-86adddeb93e1',
 'f9ccb86d-d322-4b00-8cf8-3884a587a135',
 '6e2003d9-e45a-

In [23]:
vector_store.add_documents(
    documents=documents,ids=uuids
)

['b4c603f1-ee8e-47ae-bbcd-79f28d5f80f0',
 'be68792b-7095-4e0c-9e18-e0f1c46e4833',
 'bf2ee7de-b543-42a4-86a1-d093bad05a9e',
 '8338595c-bb56-4fc8-9f5c-84f7841a9447',
 '50d7483c-a6dd-4fab-8fcf-03d8c1006fca',
 '3c954fb5-3fc3-4184-b232-7ab1337b8d6b',
 '2bb456c4-c006-43ee-b09a-73f151d730b3',
 'd7b63b77-5e69-4a56-9e21-00b8af4cfbb8',
 '092285fc-5c8b-4f26-a9c5-c84fd28474af',
 'ea7b5db7-6897-462a-b814-22961fce2dfe',
 '1897acc3-0295-4e3e-a8fc-b0ba5bc8362e',
 '6317e664-f290-484b-8542-5967de8a03b5',
 '64d8ce14-e3f1-43c0-915b-48afb6b0ee78',
 'eb50ba54-94e1-4d08-a2f2-f2dd697bcc7d',
 '5f1f339d-8bf4-44b9-9c18-d14a2d16b7ff',
 'bfb58ed8-c294-45a1-a864-48bab3c9b676',
 '079babef-e749-43ba-b39a-24cbfa2d6712',
 '2529bf97-3399-4125-b2b3-7ec0828a5085',
 '716386fc-713b-4edc-a9e4-2a7af48f7917',
 '7cd266cf-ee3a-439c-8145-eda98b6e0255',
 '4143a0f5-b3f8-4c5c-bee3-0eb0442d170b',
 '16be6787-87e7-4830-9741-d3cf887cd266',
 'dd9aa3fc-732d-46bc-964f-6a6bc10094bb',
 '9f6e3870-7b76-4161-9624-96a3102e93da',
 '6edcb0ca-36e7-

In [24]:
vector_store.similarity_search(
    query="What is the weather like tomorrow?", k=2
)

[Document(id='bcdfb34d-d160-452f-a283-7ba2d2d598ed', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(id='6ebe3f4d-19d8-4469-b224-327a82f25203', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.')]

In [25]:
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={'k': 2,'score_threshold':0.5}
)

In [26]:
retriever.invoke("what is llama model?")

[Document(id='7a166af3-dcfa-4249-9617-8677db2fd83d', metadata={'author': 'monal singh', 'containsaigeneratedcontent': 'Yes', 'creationdate': '2025-01-30T20:27:03+00:00', 'creator': 'Canva', 'keywords': 'DAGdmhcqnYw,BAEmsmap8Lg,0', 'moddate': '2025-01-30T20:26:59+00:00', 'page': 31.0, 'page_label': '32', 'producer': 'Canva', 'source': 'D:\\PROJECT_PRACTISE_DIRS\\AGENTICAI\\2-Langchain_Basics\\2.2-DataTransformer\\syllabus.pdf', 'title': 'Ultimate Data Science & GenAI Bootcamp', 'total_pages': 34.0}, page_content='Databases for RAG (e.g., Pinecone,\nFAISS, Chroma DB)\nRole of LLMs in RAG How LLMs (Large Language Models)\nEnhance Generation in RAG, Fine-\nTuning LLMs for Retrieval-Augmented\nTasks'),
 Document(id='6168ca24-9658-467f-839a-7fb406e8fe08', metadata={'author': 'monal singh', 'containsaigeneratedcontent': 'Yes', 'creationdate': '2025-01-30T20:27:03+00:00', 'creator': 'Canva', 'keywords': 'DAGdmhcqnYw,BAEmsmap8Lg,0', 'moddate': '2025-01-30T20:26:59+00:00', 'page': 17.0, 'page_la

In [27]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")

In [28]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
    

In [29]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [30]:
rag_chain.invoke("for breakfast this morning.")

'I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'